In [1]:
# Data science
import pandas as pd

# API
import requests
import json

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func

In [2]:
# Read the CSV files
df1 = pd.read_csv('Resources/parks.csv')
df = pd.read_csv('Resources/species.csv')

In [3]:
df1.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [4]:
df.head()

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN


In [5]:
#drop_list = ["Record Status", "Occurrence", "Nativeness", "Abundance", "Seasonality"]
df.drop(["Record Status", "Occurrence", "Nativeness", "Abundance", "Seasonality"], axis=1, inplace=True)

In [6]:
df.head()

,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Conservation Status
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Species of Concern
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Endangered
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",NaN


In [ ]:
# Create engine for connection to the SQLite
engine = create_engine('sqlite:///national_parks.sqlite')

# Save the dataframes to the database
df1.to_sql('parks', engine, index=False, if_exists='replace')
df.to_sql('species', engine, index=False, if_exists='replace')

In [ ]:
# Read the data from the database
with engine.connect() as connection:
    parks_table = pd.read_sql('SELECT * FROM parks', connection)
    species_table = pd.read_sql('SELECT * FROM species', connection)

species_table.head()

In [ ]:
parks_table.head(20)

In [ ]:
# Split parks with multiple states into duplicate rows with one for each state

query = """
    WITH split_states AS (
        SELECT
            p."Park Code",
            p."Park Name",
            TRIM(CASE
                WHEN INSTR(p.State, ', ') > 0 THEN SUBSTR(p.State, 1, INSTR(p.State, ', ') - 1)
                ELSE p.State
            END) AS State,
            p.Acres,
            p.Latitude,
            p.Longitude
        FROM
            parks AS p
        UNION
        SELECT
            p."Park Code",
            p."Park Name",
            TRIM(CASE
                WHEN INSTR(p.State, ', ') > 0 THEN SUBSTR(p.State, INSTR(p.State, ', ') + 1)
                ELSE ''
            END) AS State,
            p.Acres,
            p.Latitude,
            p.Longitude
        FROM
            parks AS p
        UNION
        SELECT
            p."Park Code",
            p."Park Name",
            TRIM(CASE
                WHEN INSTR(p.State, ', ') > 0 THEN SUBSTR(p.State, INSTR(p.State, ', ') + 2)
                ELSE ''
            END) AS State,
            p.Acres,
            p.Latitude,
            p.Longitude
        FROM
            parks AS p
    )
    SELECT
        s."Species ID",
        s.Category,
        s."Order",
        s.Family,
        s."Scientific Name",
        s."Common Names",
        s."Conservation Status",
        t."Park Code",
        t."Park Name",
        TRIM(t.State) AS State,
        t.Acres,
        t.Latitude,
        t.Longitude
    FROM
        split_states as t
    JOIN
        species as s ON s."Park Name" = t."Park Name"
    WHERE
        s."Conservation Status" <> 'None'
        AND TRIM(t.State) <> '';
    """


combined_df = pd.read_sql(text(query), con=engine)
combined_df

In [ ]:
# Confirm that states are split
combined_df.loc[combined_df["Park Name"] == "Death Valley National Park"].reset_index(drop=True)

In [ ]:
# Save the dataframe to the database
combined_df.to_sql('combined', engine, index=False, method='multi', if_exists='replace')

In [ ]:
# Create engine for conection to the SQLite
engine = create_engine('sqlite:///national_parks.sqlite')

with engine.connect() as connection:
    combined_table = pd.read_sql('SELECT * FROM parks', connection)

In [ ]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

In [ ]:
print(combined_df["State"].nunique())
combined_df.State.sort_values().unique()

In [ ]:
print(combined_df["Conservation Status"].nunique())
combined_df["Conservation Status"].unique()

In [ ]:
# Q3: How does the size of different parks compare with the conservation status of the animals in each park across the country?

In [ ]:
query = """
    SELECT
        "Park Name",
        Latitude,
        Longitude,
        "Conservation Status",
        COUNT ("Conservation Status") AS "Species Count",
        State,
        Acres
    FROM
        combined
    WHERE
        "Conservation Status" IN ('Endangered', 'Threatened', 'Species of Concern', 'Under Review')
    GROUP BY 
        "Park Name", "State", "Conservation Status"
    ORDER BY
        Acres DESC
    """

q3_df = pd.read_sql(text(query), con=engine)
q3_df.head(15)

In [ ]:
q3_df.info()

In [ ]:
q3_df.value_counts("Conservation Status")

In [ ]:
# Total Species count per park
df3 = q3_df.groupby("Park Name")["Species Count"].sum()
df3 = df3.sort_values(ascending=False)
df3

In [ ]:
# df_split = q3_df.assign(State=q3_df["State"].str.split(", ")).explode("State")

# df_split = df_split.reset_index(drop=True)

# df_split.head(20)

In [ ]:
# Save the dataframes to the database
q3_df.to_sql('conservation', engine, index=False, if_exists='replace')

In [ ]:
# engine.dispose()